In [5]:
import os
import re
import aocd
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
from aocd import submit

#Part 1
rdata = aocd.get_data(day=16, year=2023).split('\n')
#with open('example_input.txt') as f:
#    rdata = f.readlines()


In [6]:
data = [line.strip() for line in rdata]
data = [line.replace('\\','>') for line in data]
data = np.array([[y for y in line] for line in data])
data

array([['>', '.', '.', ..., '.', '.', '.'],
       ['.', '/', '.', ..., '.', '.', '.'],
       ['|', '.', '.', ..., '/', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [40]:
# keep track of each laser point and direction at each timestep
# keep a boolean for if we have visited this position in this direction before...


directions = dict(u = (-1,0),
                    d = (1,0),
                    r = (0,1),
                    l = (0,-1))
def add(a,b):
    return tuple(sum(i) for i in zip(a,b))


def go_left(pos, start_dir):
    return add(pos, directions['l']), 'l'
def go_right(pos, start_dir):
    return add(pos, directions['r']), 'r'
def go_up(pos, start_dir):
    return add(pos, directions['u']), 'u'
def go_down(pos, start_dir):
    return add(pos, directions['d']), 'd'
def no_change(pos, start_dir):
    return add(pos, directions[start_dir]), start_dir

def is_outside(grid, position):
    if (position[0] < 0) or (position[1] < 0):
        return True
    if (position[0] >= grid.shape[0]) or (position[1] >= grid.shape[1]):
        return True



def get_no_energized(heads):
    visited = dict(u = np.zeros_like(data, dtype=bool),
                d = np.zeros_like(data, dtype=bool),
                r = np.zeros_like(data, dtype=bool),
                l = np.zeros_like(data, dtype=bool))
    i = 0
    while len(heads):
        #print(i, heads)
        position, start_direction = heads.pop()
        if is_outside(data, position):
            #outside map
            continue

        # if we have already been in this position and direction, drop head
        if visited[start_direction][position]:
            continue

        obstacle = data[position]
        if obstacle == '.':
            new_position, new_direction = no_change(position, start_direction)
            heads += [(new_position, new_direction)]
        elif (obstacle == '-') & (start_direction in ['l','r']):
            new_position, new_direction = no_change(position, start_direction)
            heads += [(new_position, new_direction)]
        elif (obstacle == '|') & (start_direction in ['u','d']):
            new_position, new_direction = no_change(position, start_direction)
            heads += [(new_position, new_direction)]
        elif (obstacle == '-') & (start_direction in ['u','d']):
            # split beam
            new_position, new_direction = go_left(position, start_direction)
            heads += [(new_position, new_direction)]
            new_position, new_direction = go_right(position, start_direction)
            heads += [(new_position, new_direction)]
        elif (obstacle == '|') & (start_direction in ['l','r']):
            # split beam
            new_position, new_direction = go_up(position, start_direction)
            heads += [(new_position, new_direction)]
            new_position, new_direction = go_down(position, start_direction)
            heads += [(new_position, new_direction)]
        elif (obstacle == '/'):
            if start_direction == 'u':
                new_position, new_direction = go_right(position, start_direction)
            elif start_direction == 'l':
                new_position, new_direction = go_down(position, start_direction)
            elif start_direction == 'r':
                new_position, new_direction = go_up(position, start_direction)
            elif start_direction == 'd':
                new_position, new_direction = go_left(position, start_direction)
            heads += [(new_position, new_direction)]
        elif (obstacle == '>'): # \
            if start_direction == 'u':
                new_position, new_direction = go_left(position, start_direction)
            elif start_direction == 'l':
                new_position, new_direction = go_up(position, start_direction)
            elif start_direction == 'r':
                new_position, new_direction = go_down(position, start_direction)
            elif start_direction == 'd':
                new_position, new_direction = go_right(position, start_direction)
            heads += [(new_position, new_direction)]
        else:
            wtf


        visited[start_direction][position] = True
        i += 1
    visited_full = visited['u'] | visited['d'] | visited['r'] | visited['l']
    return visited_full.sum()
position = (0,0)
start_direction = "r"
heads = [(position, start_direction)]
get_no_energized(heads)

7046

In [38]:
#part 2
import itertools 

def combine(rows, columns, direction):
    combined = itertools.product(rows, columns)
    combined = itertools.product(combined, np.array([direction]))
    return list(combined)

up_starts = combine(rows = np.array([0]),
                    columns = np.arange(110),
                    direction = 'd')
down_starts = combine(rows = np.array([109]),
                    columns = np.arange(110),
                    direction = 'u')
left_starts = combine(rows = np.arange(110),
                    columns = np.array([0]),
                    direction = 'r')
right_starts = combine(rows = np.arange(110),
                    columns = np.array([109]),
                    direction = 'l')

potential_starts = up_starts + down_starts + left_starts + right_starts
potential_starts

energized = []

for start in tqdm(potential_starts):
    heads = [start]
    energized += [get_no_energized(heads)]
max(energized)

100%|██████████| 440/440 [00:07<00:00, 62.78it/s]


7313